In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from dotenv import load_dotenv

load_dotenv()


True

In [23]:
import dspy

llama3b = dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p2-3b-instruct', temperature=0.7)
gpt4o = dspy.LM('openai/gpt-4o', temperature=0.7)

dspy.configure(lm=llama3b)

In [5]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

Generating test split: 100%|██████████| 4000/4000 [00:00<00:00, 68392.45 examples/s]


In [6]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Miss Potter', 'Chris Noonan', 'Academy Award for Best Director']


In [7]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [8]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [9]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [19]:
react(claim="David Gregory was born in 1625.").titles[:3]

[]

Trace(request_id=a2a603dfc6e848cb8b43a1e13551ede7)

In [20]:
dspy.inspect_history(n=1)





[2025-03-12T18:38:23.559367]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Find all Wikipedia titles relevant to verifying (or refuting) the claim.


User message:

[[ ## claim ## ]]
David Gregory was born in 1625.

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
Searching for information about David Gregory's birth.

[[ ## tool_name_0 ## ]]
search_wikipedia

[[ ## tool_args_0 ## ]]
{"query": {"type": "string"}}

[[ ## observation_0 ## ]]
Failed to execute: Arg quer

In [21]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [24]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 25.00 / 100 (25.0%): 100%|██████████| 100/100 [04:23<00:00,  2.63s/it]

2025/03/12 18:43:58 INFO dspy.evaluate.evaluate: Average Metric: 24.99999999999999 / 100 (25.0%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Trinity Episcopal Church (Houghton, Michigan), S...","{'thought_0': ""The claim mentions the Church of England and the Tr...",The claim mentions the Church of England and the Trinity Episcopal...,"['John Lettice', 'John Kettlewell', 'John Purchas', 'Joachim Lütke...",
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]","{'thought_0': 'The french fighter is likely Patrick Surtain', 'too...",The reasoning is that the tool was unable to find reliable informa...,"['Billy Stewart', 'Jean-Patrick Lescarboura', 'Simon Patrick Stewa...",
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Fernand Rivers, Ed Wood, Glen or Glenda]",{'thought_0': 'The writer/director/actor from Glen or Glenda and F...,The writer/director/actor from Glen or Glenda and Fernand Rivers i...,['The Incredibly Strange Creatures Who Stopped Living and Became M...,
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Sandi Sissel, Chicken Ranch (film), The End of Suburbia]",{'thought_0': 'Search for the director of The End of Suburbia to s...,The director of The End of Suburbia may have also directed the fil...,"['Sandi Sissel', 'Escape from Suburbia', 'Suburb (1951 film)', 'Su...",✔️ [0.333]
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Christopher Walken, Peter Pan Live!]",{'thought_0': 'The actor who played captain hook in the live produ...,The reasoning behind this response is that none of the provided to...,"['Cyril Ritchard', 'Captain James Hook', 'E. Holman Clark', 'Peter...",


25.0

[Trace(request_id=875a9051f7e749f1ba7aaf8200dc1ac6), Trace(request_id=090ffb3a1a5b45108fb5aa1f07b1716d), Trace(request_id=91027bc83e214a169221ae7400cb04b9), Trace(request_id=be4ed3e01bc145deb3415efde21b19b2), Trace(request_id=74c2819f278844c1ac0d27ca45e7f14d), Trace(request_id=cc1665a7fcfc431ca5e358e59a90b3bc), Trace(request_id=a9443893823f4e51b39714f803cfcf2e), Trace(request_id=3a130543ee004d7daa6a77c5310a76ee), Trace(request_id=0c3a3ffebb394a90af54edf918e1df9b), Trace(request_id=5529cbd922d54eb7882406c5394966cc)]

In [25]:
dspy.inspect_history(n=1)





[2025-03-12T18:43:58.296078]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Find all Wikipedia titles relevant to verifying (or refuting) the claim.


User message:

[[ ## claim ## ]]
The director of Daughters of Mara's first album is the son of the producer who worked for the group Badfinger.

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
The director of Daughters of Mara's first album is the son of the producer who worked for the group Badfinger. This might be a 

In [29]:
kwargs = dict(teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o, max_errors=999)

tp = dspy.MIPROv2(metric=top5_recall, auto="medium", num_threads=16, **kwargs)
optimized_react = tp.compile(react, trainset=trainset, max_bootstrapped_demos=3, max_labeled_demos=0)

2025/03/12 18:47:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 9
valset size: 80

2025/03/12 18:47:43 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/12 18:47:43 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/12 18:47:43 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=9 sets of demonstrations...


Bootstrapping set 1/9
Bootstrapping set 2/9


 35%|███▌      | 7/20 [01:45<03:15, 15.03s/it]


Bootstrapped 3 full traces after 7 examples for up to 1 rounds, amounting to 7 attempts.
Bootstrapping set 3/9


 35%|███▌      | 7/20 [01:16<02:21, 10.90s/it]


...

2025/03/12 19:10:05 INFO dspy.evaluate.evaluate: Average Metric: 11.000000000000002 / 25 (44.0%)
2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 44.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 3', 'Predictor 1: Few-Shot Set 8'].
2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [42.67, 45.33, 30.67, 44.0, 44.0, 33.33, 30.67, 42.67, 34.67, 34.67, 29.33, 42.67, 42.67, 38.67, 34.67, 29.33, 44.0, 41.33, 45.33, 40.0, 42.67, 40.0, 44.0]
2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [27.92, 40.83, 32.5]
2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 40.83
2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/12 19:10:05 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 27 / 28 - Minibatch ==



Average Metric: 8.67 / 25 (34.7%): 100%|██████████| 25/25 [01:18<00:00,  3.16s/it]

2025/03/12 19:11:24 INFO dspy.evaluate.evaluate: Average Metric: 8.666666666666668 / 25 (34.7%)
2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 34.67 on minibatch of size 25 with parameters ['Predictor 0: Instruction 3', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 7', 'Predictor 1: Few-Shot Set 6'].
2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [42.67, 45.33, 30.67, 44.0, 44.0, 33.33, 30.67, 42.67, 34.67, 34.67, 29.33, 42.67, 42.67, 38.67, 34.67, 29.33, 44.0, 41.33, 45.33, 40.0, 42.67, 40.0, 44.0, 34.67]
2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [27.92, 40.83, 32.5]
2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 40.83
2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/12 19:11:24 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 28 / 28 - Full Evaluation =====
2025/03/


Average Metric: 33.00 / 80 (41.2%): 100%|██████████| 80/80 [00:46<00:00,  1.73it/s]

2025/03/12 19:12:11 INFO dspy.evaluate.evaluate: Average Metric: 33.0 / 80 (41.2%)
2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 41.25
2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [27.92, 40.83, 32.5, 41.25]
2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 41.25
2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/12 19:12:11 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 41.25!


In [30]:
evaluate(optimized_react)

Average Metric: 40.33 / 100 (40.3%): 100%|██████████| 100/100 [01:16<00:00,  1.31it/s]

2025/03/12 20:11:03 INFO dspy.evaluate.evaluate: Average Metric: 40.33333333333333 / 100 (40.3%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Trinity Episcopal Church (Houghton, Michigan), S...",{'thought_0': 'The claim mentions a Sussex clergyman who wrote dev...,The claim states that the Church of England's movement that inspir...,"[Oxford Movement, Sussex clergyman devotional literature Trinity E...",✔️ [0.333]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]",{'thought_0': 'The claim mentions a French fighter trained by Bobb...,The claim mentions a French fighter trained by Bobby Stewart. The ...,"[Daniel Cormier, Bobby Stewart, French fighter]",✔️ [0.333]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Fernand Rivers, Ed Wood, Glen or Glenda]",{'thought_0': 'The claim requires finding information about the ca...,The claim states that Ed Wood Jr. and Fernand Rivers share the car...,"[writer, producer, director]",
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Sandi Sissel, Chicken Ranch (film), The End of Suburbia]",{'thought_0': 'The claim that the film by Sandi Sissel was release...,The claim states that the film by Sandi Sissel was released before...,[],
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Christopher Walken, Peter Pan Live!]",{'thought_0': 'The search results indicate that there is no clear ...,The claim states that the actor who played Captain Hook in a live ...,"[Cyril Ritchard, John Cazale, Taylor Louderman, The Deer Hunter, H...",✔️ [0.333]


40.33

[Trace(request_id=0d6b27fc66564be7a15962e63abe610f), Trace(request_id=8a19f5b5f7ee4311ba76bdcec49ad6d0), Trace(request_id=877681ac6b8b434d91f441e62018f076), Trace(request_id=080ee1f284ed47b3a02fb66d845ab64f), Trace(request_id=2fdd0ccbdb904cc8b317f1e0ff68b27b), Trace(request_id=73f9b24734ca482f8f95fbd2a342715b), Trace(request_id=b175ad39c30a46deacc86debdb77d742), Trace(request_id=a63346938eea485296c8bb247854f112), Trace(request_id=5b9b7949a9b24ddc94830d5d57839a2c), Trace(request_id=317fb5e4c168425b927553f028ca336d)]

In [31]:
dspy.inspect_history(n=1)





[2025-03-12T20:11:03.541089]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Identify and gather Wikipedia titles that are essential for assessing the validity of the given claim. Your process should involve step-by-step reasoning to determine the relevant entities and relationships within the claim, utilizing available tools to search and retrieve necessary information. Conclude with a comprehensive list of Wikipedia titles that serve as evidence for either suppor

In [33]:
optimized_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Joe Orton', 'Bernard-Marie Koltès', 'Arthur Miller']

Trace(request_id=ed7f03d0960243d3a5063086a17b9642)

In [35]:
react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles[:3]

['Joe Orton', 'Bernard-Marie Koltès', 'Jean-Jacques Bernard']

Trace(request_id=6c1f25bacf42457698c347a05499e39e)

In [36]:
optimized_react.save("optimized_react.json")

loaded_react = dspy.ReAct("claim -> titles: list[str]", tools=[search_wikipedia, lookup_wikipedia], max_iters=20)
loaded_react.load("optimized_react.json")

loaded_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Joe Orton', 'Bernard-Marie Koltès', 'Arthur Miller']

Trace(request_id=7bd99a67e4024484837a103ec4c969f8)